<a href="https://colab.research.google.com/github/yoavdana/ex1_DL/blob/main/ex1_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/yoavdana/ex1_DL.git

fatal: destination path 'ex1_DL' already exists and is not an empty directory.


In [4]:
import torch
import numpy as np
import scipy.stats as si
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import time
import matplotlib.pyplot as plt
INPUT_DIM = 9 * 20
OUTPUT_DIM = 2



In [5]:
SEQ_LENGTH=20
BOOTSTRAP_SIZE=24000
NUMBER_OF_BATCHS=5



def data_to_input(sequence, pos_or_neg):
    mapping = {'A': 0, 'R': 1, 'N': 2, 'D': 3, 'C': 4, 'E': 5, 'Q': 6, 'G': 7, 'H': 8, 'I': 9, 'L': 10, 'K': 11,
               'M': 12, 'F': 13, 'P': 14, 'S': 15, 'T': 16, 'W': 17, 'Y': 18, 'V': 19}
    map=np.zeros((9, 20))
    for i, seq in enumerate(sequence):
        map[i,mapping[seq]]+=1
    map = map.flatten()
    return np.concatenate([map, np.array([pos_or_neg])])


def Read_Data(filename, pos_or_neg):
    file = open(filename, 'r')
    lines=file.readlines()
    if pos_or_neg==1:
        DATA=np.zeros((len(lines), 181))
        for i, line in enumerate(lines):
            input = data_to_input(line.replace('\n', ''), pos_or_neg)
            DATA[i] = input
        DATA=bootstrap(DATA, BOOTSTRAP_SIZE, NUMBER_OF_BATCHS)
    else:
        DATA = np.zeros((len(lines), 181))
        for i, line in enumerate(lines):
            input = data_to_input(line.replace('\n', ''), pos_or_neg)
            DATA[i] = input
    return DATA


def bootstrap(DATA,size,NUMBER_OF_BATCHS):

    new_DATA=np.zeros((size,181))
    N=DATA.shape[0]
    batch_size=N//NUMBER_OF_BATCHS
    for i in range(NUMBER_OF_BATCHS):

        random = np.random.randint(batch_size*i,batch_size*(i+1), size=size//NUMBER_OF_BATCHS)
        new_DATA[((size//NUMBER_OF_BATCHS)*i):(size//NUMBER_OF_BATCHS)*(i+1), :] = DATA[random, :]
    return new_DATA


def DATA_pre_pros(filename_pos,filename_neg):

    neg_data=Read_Data(filename_neg, 0)
    pos_data=Read_Data(filename_pos, 1)
    final_data = np.concatenate([neg_data, pos_data])
    np.random.shuffle(final_data)
    train_set = final_data[:int(len(final_data)*0.9)]
    test_set = final_data[int(len(final_data)*0.9):]
    return train_set, test_set



def shuffle_data(data_Xy):
    np.random.shuffle(data_Xy)
    return data_Xy[:,:180],data_Xy[:,-1]


In [6]:
INPUT_1=100
INPUT_2=50
INPUT_3=10
P_DROPOUT=0.25


class NetWork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.input_fc = nn.Linear(input_dim, INPUT_1)
        
        self.hidden_1_fc = nn.Linear(INPUT_1, INPUT_2)
        
        self.hidden_2_fc = nn.Linear(INPUT_2, INPUT_3)
        
        self.output_fc = nn.Linear(INPUT_3, output_dim)

        self.dropout = nn.Dropout(P_DROPOUT)
        

    def forward(self, x):
   

        batch_size = x.shape[0]


        h_1 = F.relu(self.input_fc(x))

        h_1=self.dropout(h_1)
       

        h_2 = F.relu(self.hidden_1_fc(h_1))


        h_3 = F.relu(self.hidden_2_fc(h_2))


        y_pred = self.output_fc(h_3)

       
        
        return y_pred, h_2


In [7]:

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc


def train(model, iterator_x, iterator_y, optimizer, criterion, device):
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for i, x in enumerate(iterator_x):
        x = x.to(device)
        y = iterator_y[i].to(device)

        optimizer.zero_grad()

        y_pred, _ = model(x)

        loss = criterion(y_pred, y)

        acc = calculate_accuracy(y_pred, y)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator_x), epoch_acc / len(iterator_x)


def evaluate(model, iterator_x, iterator_y, criterion, device):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    for i, x in enumerate(iterator_x):
        x = x.to(device)
        y = iterator_y[i].to(device)

        y_pred, _ = model(x)

        loss = criterion(y_pred, y)

        acc = calculate_accuracy(y_pred, y)

 
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator_x), epoch_acc / len(iterator_x)




    

In [8]:
filename_pos='/content/ex1_DL/resorces/pos_A0201.txt'
filename_neg='/content/ex1_DL/resorces/neg_A0201.txt'

train_set, test_set=DATA_pre_pros(filename_pos,filename_neg)

# LEARNNING_RATE=0.00015

def main():
    model = NetWork(INPUT_DIM, 2)
    print(f'The model has {count_parameters(model):,} trainable parameters')
    optimizer = optim.Adam(model.parameters(),lr=LEARNNING_RATE)
    criterion = nn.CrossEntropyLoss()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    criterion = criterion.to(device)




    EPOCHS = 50

    best_valid_loss = float('inf')
    train_per_ep=[]
    test_per_ep=[]
    test_acc = 0
    for epoch in range(EPOCHS):


        train_x,train_y=shuffle_data(train_set)
        train_iterator_x = torch.from_numpy(train_x.astype('float32')).split(64)
        train_iterator_y = torch.from_numpy(train_y.astype('int64')).split(64)

        test_x,test_y=shuffle_data(test_set)
        test_iterator_x = torch.from_numpy(test_x.astype('float32')).split(64)
        test_iterator_y = torch.from_numpy(test_y.astype('int64')).split(64)

        train_loss, train_acc = train(model, train_iterator_x, train_iterator_y, optimizer, criterion, device)

        test_loss, test_acc = evaluate(model, test_iterator_x, test_iterator_y,criterion, device)
        train_per_ep.append(train_loss)
        test_per_ep.append(test_loss)
        
        print(f'Epoch: {epoch + 1:02}')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
        print(f'Epoch: {epoch + 1:02}')
        print(f'\tTest Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%')

    train_per_ep=np.array(train_per_ep)
    test_per_ep=np.array(test_per_ep)
    epocs=np.arange(1,EPOCHS+1)
    # plt.figure()
    # plt.plot(epocs,train_per_ep)
    # plt.plot(epocs,test_per_ep)
    # plt.legend(['train loss','test loss'])
    # plt.show()
    return test_acc, np.mean(np.abs(np.array(train_per_ep) - np.array(test_per_ep)))

In [ ]:
learning_rate = np.linspace(0.0001, 0.001, num=10)
input_1 = [70, 100, 130, 150]
input_2 = [30, 40, 50, 60]
input_3 = [10, 15, 20, 25]
p = [0.2, 0.25, 0.35, 0.42, 0.5] 
acc = list()
overfit=list()
all=np.load('/content/drive/MyDrive/Colab Notebooks/opt.npy')
for rate in learning_rate:
  for input_1_ in input_1:
    for input_2_ in input_2:
      for input_3_ in input_3:
        for p_ in p:
            LEARNNING_RATE = rate
            INPUT_1 = input_1_
            INPUT_2 = input_2_
            INPUT_3 = input_3_
            P_DROPOUT = p_ 
          if np.array([INPUT_1,INPUT_2,INPUT_3,P_DROPOUT,LEARNNING_RATE]) in all[:,0:-2]:
            continue
          else:  
            ac,of = main()
            acc.append(ac)
            overfit.append(of)
            params=np.array([INPUT_1,INPUT_2,INPUT_3,P_DROPOUT,LEARNNING_RATE, ac, of])

          
          all=np.vstack([all, params])
          np.save('/content/drive/MyDrive/Colab Notebooks/opt.npy,all)
        



The model has 15,132 trainable parameters
Epoch: 01
	Train Loss: 0.529 | Train Acc: 80.54%
Epoch: 01
	Test Loss: 0.301 | Test Acc: 89.55%
Epoch: 02
	Train Loss: 0.273 | Train Acc: 89.34%
Epoch: 02
	Test Loss: 0.246 | Test Acc: 90.35%
The model has 15,132 trainable parameters
Epoch: 01
	Train Loss: 0.623 | Train Acc: 60.51%
Epoch: 01
	Test Loss: 0.349 | Test Acc: 88.53%
Epoch: 02
	Train Loss: 0.286 | Train Acc: 88.92%
Epoch: 02
	Test Loss: 0.251 | Test Acc: 90.46%
The model has 15,132 trainable parameters
Epoch: 01
	Train Loss: 0.545 | Train Acc: 72.76%
Epoch: 01
	Test Loss: 0.291 | Test Acc: 88.88%


KeyboardInterrupt: ignored

In [12]:
a=np.load('/content/drive/MyDrive/Colab Notebooks/opt.npy')
print(np.array([7.0e+01,3.0e+01,1.0e+01,2.0e-01,1.0e-04]) in a[:,0:-2])


True
